In [ ]:
import torch
import numpy as np
from glob import glob
import os
import matplotlib
import pandas as pd

PATH = '/home/michael/Dropbox/Master thesis/results_graph_dc_new_lr_0_5/FedFOLA/kld/with_reparam/Graph-DC_FedFOLA_NE_KLD_WITH_repara_multistep_lr_0_05_local_update_steps_1_csd_imp_10_kld_imp_0/FedFOLA_GraphDC_gin_on_fs_contest_data_lr0.05_lstep1_/features'

train_test_val = 'val'

save_path = '/home/michael/Dropbox/Master thesis/results_graph_dc_new_lr_0_5/FedFOLA/CSV/kld/with_reparam/feature_analysis/Graph-DC_FedFOLA_NE_KLD_WITH_repara_multistep_lr_0_05_local_update_steps_1_csd_imp_10_kld_imp_0' +'/'+ train_test_val + '/'
if train_test_val == 'train':
    nun_samples = 11649
elif train_test_val == 'val':
    nun_samples = 3883
elif train_test_val == 'test':
    nun_samples = 3889

In [ ]:
font = {'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
#interm_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
mu_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],13:[]}
logvar_trains = {1:[], 2:[],3:[], 4:[],5:[], 6:[],7:[], 8:[],9:[], 10:[],11:[], 12:[],
              13:[]}


In [ ]:
all_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, '*.pt'))]
all_files.sort()

In [ ]:
def get_files(path):
    for client in range(1, 13+1):
        client_path = f"{PATH}/client_{client}"
        all_files = [file
                 for path, subdir, files in os.walk(client_path)
                 for file in glob(os.path.join(path, '*.pt'))]
        all_files.sort()
        for file in all_files:
            file_name = os.path.basename(file)
            file_name_without_ending = file_name[:-3]
            if file_name_without_ending.startswith('mu_' + train_test_val) and not \
                    file_name_without_ending.endswith('labels'):
                mu_trains[client].append(torch.load(file).cpu().detach().numpy())
            if file_name_without_ending.startswith('std' + train_test_val) and not\
                    file_name_without_ending.endswith('labels'):
                logvar_trains[client].append(torch.load(file).cpu().detach().numpy())

        #interm_trains[client] = np.concatenate(interm_trains[client])
        mu_trains[client] = np.concatenate(mu_trains[client])
        logvar_trains[client] = np.concatenate(logvar_trains[client])



In [ ]:
PATH

In [ ]:
get_files(PATH)

In [ ]:
import numpy as np
#concat_X=global_train
data_out_logvar = {}
data_out_mu = {}

for key in logvar_trains.keys():
    data_out_logvar[key] = logvar_trains[key]
    data_out_mu[key] = mu_trains[key]

X_concat_logvar = []
X_concat_mu = []

for key in logvar_trains.keys():
    X_concat_logvar.append(data_out_logvar[key])
    X_concat_mu.append(data_out_mu[key])

X_concat_logvar = np.concatenate(X_concat_logvar)
X_concat_mu = np.concatenate(X_concat_mu)



In [ ]:
isExist = os.path.exists(save_path)
if not isExist:
    os.makedirs(save_path)

In [ ]:
val_mu = []
val_std = []
mu_trains_mean = []
std_trains_mean = []

In [ ]:
for client in range(1, 13+1):


    val_mu_curr = mu_trains[client].mean()
    val_std_curr = np.mean(np.exp(logvar_trains[client] * 0.5))

    val_mu.append(val_mu_curr)
    val_std.append(val_std_curr)

    mu_mean = np.mean(mu_trains[client], axis = 0)
    std_mean = np.mean(np.exp(logvar_trains[client] * 0.5), axis = 0)

    mu_trains_mean.append(mu_mean)
    std_trains_mean.append(std_mean)


In [ ]:
data_map = {
    ('mu_mean'): val_mu,
    ('std_mean'): val_std,
}
df = pd.DataFrame(data=data_map)
df.to_csv(save_path + 'statistics.csv')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20, 9))
plt.rcParams["figure.figsize"] = (20, 9)
plt.rc('xtick', labelsize=24)    # fontsize of the tick labels
plt.rc('ytick', labelsize=24)    # fontsize of the tick labels
plt.rc('legend', fontsize=24)
plt.rc('font', size=24)

In [ ]:
x = range(1,65)

In [ ]:
for client in range(1, 13+1):
    plt.plot(x, std_trains_mean[client-1], label = client)
plt.legend(bbox_to_anchor=(1.12, 1.0))
plt.savefig(save_path + 'std_clients')

In [ ]:
for client in range(1, 13+1):
    plt.plot(x, mu_trains_mean[client-1], label = client)
plt.legend(bbox_to_anchor=(1.12, 1.0))
plt.savefig(save_path + 'mu_clients')